<img src="https://i.ibb.co/qjt4Ymb/2022-09-19-004719.png" alt="2022-09-19-004719" border="0">

# 亞洲大學112深度學習教材(AUDL112-W10-Notebook2)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# PyTorch RNN

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
from pathlib import Path

In [ ]:
!ls  \Stock

## Data source
https://drive.google.com/file/d/1nQHtdlXFMGaUqdzW9u2jjQI7vSZOaaN2/view?usp=sharing

In [ ]:
# from chu20230205@gmail.com
%%bash
gdown https://drive.google.com/uc?id=1nQHtdlXFMGaUqdzW9u2jjQI7vSZOaaN2
unzip individual_stocks_5yr.zip
rm individual_stocks_5yr.zip

In [ ]:
path = Path('individual_stocks_5yr')
df = pd.read_csv(path/'AAPL_data.csv')

In [ ]:
df.head()

In [ ]:
df=df['close']

In [ ]:
df =(df.values).reshape(-1,1)
df = (normalize(df, axis=0)).squeeze()

In [ ]:
df1 = pd.read_csv(path/'GOOGL_data.csv')

In [ ]:
df1=df1['close']

In [ ]:
df1 =(df1.values).reshape(-1,1)

In [ ]:
df1 = (normalize(df1, axis=0)).squeeze()

In [ ]:
df1.shape

In [ ]:
df2 = pd.read_csv(path/'MSFT_data.csv')

In [ ]:
df2=df2['close']

In [ ]:
df2 =(df2.values).reshape(-1,1)
df2 = (normalize(df2, axis=0)).squeeze()

In [ ]:
df3 = pd.read_csv(path/'AMZN_data.csv')

In [ ]:
df3=df3['close']

In [ ]:
df3 =(df3.values).reshape(-1,1)
df3 = (normalize(df3, axis=0)).squeeze()

In [ ]:
df4 = pd.read_csv(path/'FB_data.csv')

In [ ]:
df4=df4['close']

In [ ]:
df4 =(df4.values).reshape(-1,1)
df4 = (normalize(df4, axis=0)).squeeze()

In [ ]:
df4[0]

In [ ]:
wind=20 # Time Window
TimeS = np.empty([len(df)-wind-1, wind+1])
TimeS.shape
TimeS=np.float32(TimeS)
TimeS1=np.float32(np.empty([len(df)-wind-1, wind+1]))
TimeS2=np.float32(np.empty([len(df)-wind-1, wind+1]))
TimeS3=np.float32(np.empty([len(df)-wind-1, wind+1]))
TimeS4=np.float32(np.empty([len(df)-wind-1, wind+1]))
type(TimeS[0][0])

In [ ]:
for i in range(len(df)-wind-1):
    TimeS[i,:]=df[i:i+wind+1]

In [ ]:
TimeS

In [ ]:
TargetNump=TimeS[:,20]

In [ ]:
TargetNump.shape

In [ ]:
Y_Train=TargetNump[0:960]

In [ ]:
Y_Test= TargetNump[960:]

In [ ]:
Y_Test= Y_Test[0:256]

In [ ]:
DataNump=TimeS[:,0:20]

In [ ]:
for i in range(len(df)-wind-1):
    TimeS1[i,:]=df1[i:i+wind+1]

In [ ]:
TimeS1=TimeS1[:,0:20]

In [ ]:
TimeS1.shape

In [ ]:
for i in range(len(df)-wind-1):
    TimeS2[i,:]=df2[i:i+wind+1]

In [ ]:
TimeS2=TimeS2[:,0:20]

In [ ]:
for i in range(len(df)-wind-1):
    TimeS3[i,:]=df3[i:i+wind+1]

In [ ]:
TimeS3=TimeS3[:,0:20]

In [ ]:
for i in range(len(df)-wind-1):
    TimeS4[i,:]=df4[i:i+wind+1]


In [ ]:
TimeS4=TimeS4[:,0:20]

In [ ]:
X_complete=np.stack((DataNump, TimeS1, TimeS2, TimeS3, TimeS4), axis=0)
X_complete.shape
TimeS4[0,:]

In [ ]:
X_complete=X_complete.transpose(2,1,0)

In [ ]:
X_Train=X_complete[:,0:960,:]

In [ ]:
X_Test=X_complete[:,960:,:]

In [ ]:
np.shape(X_Test)

In [ ]:
X_Test=X_Test[:,0:256,:]

In [ ]:
np.shape(X_Test)

In [ ]:
bs=64
numlay=2
hiddenSz=20

In [ ]:
class Model1(nn.Module):

    def __init__(self):
        super(Model1, self).__init__()
        #self.rnn = nn.RNN(input_size=1,hidden_size=20, num_layers=1, batch_first = True)
        self.lstm = nn.LSTM(input_size=5,hidden_size=hiddenSz, num_layers=numlay, batch_first = True)
        self.out = nn.Linear(20,1)
        self.bn = nn.BatchNorm1d(20)
#     def forward(self,x, h_state):
#         r_out, h_state = self.rnn(x, h_state)
    def forward(self,x, h_state, c_state):
        r_out, (h_state,c_state) = self.lstm(x, (h_state,c_state))
        r_out = self.bn(r_out)

        outs=[]
        for time_step in range(r_out.size(1)):
            outs.append(self.out(r_out[:,time_step,:]))


        return torch.stack(outs,dim=1), (h_state, c_state)

In [ ]:
net=Model1()

In [ ]:
net.cuda()

In [ ]:
h_state = torch.zeros(numlay,bs,hiddenSz).cuda()
c_state = torch.zeros(numlay,bs,hiddenSz).cuda()

In [ ]:
class StockDataset(Dataset):

    def __init__(self, X_dataset, Y_dataset):

        self.X_dataset = torch.from_numpy(X_dataset)
        self.Y_dataset = torch.from_numpy(Y_dataset)

    def __len__(self):
        return np.shape(self.Y_dataset)[0]

    def __getitem__(self, idx):

        sample = {'Sequence': self.X_dataset[:,idx,:], "Target": self.Y_dataset[idx]}


        return sample

In [ ]:
Train_ds=StockDataset(X_Train, Y_Train)

In [ ]:
Train_ds[0]

In [ ]:
len(Train_ds)

In [ ]:
for i in range(len(Train_ds)):
    sample = Train_ds[i]

    print(i, sample['Sequence'].size(), sample['Target'])

    if i == 3:
        break


In [ ]:
Train_dl = DataLoader(Train_ds, batch_size=bs, shuffle=False, num_workers=4)

In [ ]:
Train_dl.dataset[2]

In [ ]:
import torch.optim as optim

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.00005)


In [ ]:
for epoc in range (100):
        running_loss = 0.0
        predictionplot=np.empty([960])

        for i, data in enumerate(Train_dl):
        # get the inputs
            inputs, labels = data['Sequence'], data['Target']

            x = Variable(inputs).cuda()
            y = Variable(labels).cuda()

        # zero the parameter gradients
            optimizer.zero_grad()

        # forward + backward + optimize
            prediction, (h_state, c_state) = net(x, h_state,c_state)
            prediction=prediction[:,19,:].squeeze()
            h_state = Variable(h_state.data)
            c_state = Variable(c_state.data)
            loss = criterion(prediction, y)
            loss.backward()
            optimizer.step()

        # print statistics
            running_loss += loss.item()
            if i % 2 == 1:    # print every 2 mini-batches
                print('[%d, %1d] loss: %.7f' %(epoc + 1, i + 1, running_loss / 2))
                running_loss = 0.0

            predictionplot[i*64:i*64+64] = prediction.cpu().detach().numpy()

print('Finished Training')

In [ ]:
 # plotting
plt.plot(np.arange(0.0, 960, 1.0), Y_Train, 'r-')
#plt.draw(); plt.pause(0.05)
#plt.plot(np.arange(0.0, 64.0, 1.0), prediction.data.numpy().flatten(), 'b-')
plt.plot(np.arange(0.0, 960, 1.0), np.asarray(predictionplot), 'b-')
plt.draw(); plt.pause(0.05)

In [ ]:
Test_ds=StockDataset(X_Test, Y_Test)

In [ ]:
Test_ds[77]

In [ ]:
Test_dl = DataLoader(Test_ds, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
with torch.no_grad():
    predictionplot=[]

    for data in Test_dl:

        inputs, labels = data['Sequence'].cuda(), data['Target'].cuda()

        # forward + backward + optimize

        prediction, (h_state,c_state) = net(inputs, h_state,c_state)
        prediction=prediction[:,19,:].squeeze()

        loss = criterion(prediction, labels)
        running_loss += loss.item()

        print('loss: %.7f' %(running_loss))
        running_loss = 0.0
        predictionplot[i*(len(prediction.cpu().numpy())):i*(len(prediction.cpu().numpy()))+len(prediction.cpu().numpy())] = prediction.cpu().detach().numpy()





In [ ]:
plt.plot(np.arange(0.0, 256, 1.0), Y_Test, 'r-')
plt.plot(np.arange(0.0, 256, 1.0), np.asarray(predictionplot), 'b-')
plt.draw();